In [62]:
import gymnasium as gym
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from IPython import display

In [63]:
env = gym.make('CartPole-v1')

In [64]:
device = torch.device('cuda')

In [65]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

In [66]:
class ReplayMemory(object):
    
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)
    
    def push(self, *args):
        self.memory.append(Transition(*args))
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [67]:
class DQN(nn.Module):
    
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)
        
    def forward(self, x):
        x = F.relu(self.layer2(F.relu(self.layer1(x))))
        return self.layer3(x)

In [68]:
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

In [69]:
env.action_space

Discrete(2)

In [70]:
n_actions = env.action_space.n
n_actions

2

In [71]:
state, info = env.reset()
state, info

(array([ 0.03015089, -0.03381533, -0.01129236,  0.03766716], dtype=float32),
 {})

In [72]:
n_observations = len(state)
n_observations

4

In [73]:
policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

<All keys matched successfully>

In [74]:
optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)

In [75]:
steps_done = 0
episode_durations = []

In [92]:
torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

tensor([[0]], device='cuda:0')

In [99]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1 * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1,1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

In [77]:
def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training..')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    
    if len(durations_t) >= 100:
        means = durations_t.unfold(0,100,1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())
        
    plt.pause(0.001)
    
    if show_result:
        display.display(plt.gcf())
    else:
        display.display(plt.gcf())
        display.clear_output(wait=True)

In [97]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(2)
    print('transitions:', transitions)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))
    print('batch:', batch)
    
    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    
    print(state_batch)
    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)
    
    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]
        
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
    # Computer Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [100]:
%%time
if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 50

for i_episode in range(num_episodes):
    # Initialize the environment and get it's state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()
        break

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

transitions: [Transition(state=tensor([[ 0.0133, -0.3993, -0.0256,  0.5576]], device='cuda:0'), action=tensor([[0]], device='cuda:0'), next_state=tensor([[ 0.0053, -0.5941, -0.0144,  0.8422]], device='cuda:0'), reward=tensor([1.], device='cuda:0')), Transition(state=tensor([[-0.0660, -1.3403,  0.1464,  2.1349]], device='cuda:0'), action=tensor([[0]], device='cuda:0'), next_state=tensor([[-0.0928, -1.5366,  0.1891,  2.4690]], device='cuda:0'), reward=tensor([1.], device='cuda:0'))]
batch: Transition(state=(tensor([[ 0.0133, -0.3993, -0.0256,  0.5576]], device='cuda:0'), tensor([[-0.0660, -1.3403,  0.1464,  2.1349]], device='cuda:0')), action=(tensor([[0]], device='cuda:0'), tensor([[0]], device='cuda:0')), next_state=(tensor([[ 0.0053, -0.5941, -0.0144,  0.8422]], device='cuda:0'), tensor([[-0.0928, -1.5366,  0.1891,  2.4690]], device='cuda:0')), reward=(tensor([1.], device='cuda:0'), tensor([1.], device='cuda:0')))
tensor([[ 0.0133, -0.3993, -0.0256,  0.5576],
        [-0.0660, -1.3403

IndexError: The shape of the mask [2] at index 0 does not match the shape of the indexed tensor [128] at index 0